In [1]:
param_market = 'AU'

from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./menu.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_item_number = df['Item_Number'].tolist()

print('总数量：' + str(len(list_item_number)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for item_number in list_item_number:
    work.put_nowait(item_number)

def crawler():
    global df
    
    while not work.empty():
        item_number = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                if param_market == 'US':
                    url = 'https://www.ebay.com/itm/' + item_number + '?_ul=US&_stpos=91710&orig_cvip=true'
                elif param_market == 'AU': 
                    url = 'https://www.ebay.com.au/itm/' + item_number + '?_ul=AU&_stpos=2019&orig_cvip=true'

                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                js = json.loads(resp.split('''$MC=(window.$MC||[]).concat(''')[1].split(''']}]})</script></body></html>''')[0] + ']}]}')
                
                # = = = = = = = = = = = = = = =
                
                market_id = js['o']['g']['marketplaceId'].strip()

                # = = = = = = = = = = = = = = =
                
                country = js['o']['g']['country'].strip()
                
                # = = = = = = = = = = = = = = =
                
                language = js['o']['g']['language'].strip()
                
                # = = = = = = = = = = = = = = =
                
                locale = js['o']['g']['locale'].strip()
                
                # = = = = = = = = = = = = = = =
                
                item_id = js['o']['g']['itemId'].strip()
                
                # = = = = = = = = = = = = = = =
                
                if item_id != item_number:
                    continue
                
                # = = = = = = = = = = = = = = =
                
                specification = js['o']['w'][0][2]['model']['modules']['ABOUT_THIS_ITEM']['sections']['features']['dataItems']
                
                # = = = = = = = = = = = = = = =
                
                if 'VEHICLE_SELECTION' in js['o']['w'][0][2]['model']['modules']:
                    category_id = js['o']['w'][0][2]['model']['modules']['VEHICLE_SELECTION']['callToActions'][0]['action']['params']['categoryId'].strip()
                    vehicle_page = js['o']['w'][0][2]['model']['modules']['COMPATIBILITY_TABLE']['paginatedTable']['pagination']['itemsPerPage']['label']['textSpans'][0]['text'].split('of')[1].strip()
                else:
                    category_id = ''
                    vehicle_page = ''
                    
                # = = = = = = = = = = = = = = =
                
                title = js['o']['w'][0][2]['model']['modules']['TITLE']['mainTitle']['textSpans'][0]['text'].strip()

                # = = = = = = = = = = = = = = =
                
                if 'subTitle' in js['o']['w'][0][2]['model']['modules']['TITLE']:
                    subtitle = js['o']['w'][0][2]['model']['modules']['TITLE']['subTitle']['textSpans'][0]['text'].strip()
                else:
                    subtitle = ''
                # = = = = = = = = = = = = = = =
                
                price = js['o']['w'][0][2]['model']['modules']['BUY_BOX']['binModel']['price']['textSpans'][0]['text'].strip()

                # = = = = = = = = = = = = = = =
                
                if 'QUANTITY' in js['o']['w'][0][2]['model']['modules']:
                    available = ''
                    sold = ''
                    list_text = js['o']['w'][0][2]['model']['modules']['QUANTITY']['availabilitySignal']['textSpans']
                    for text in list_text:
                        if 'available' in text['text']:
                            available = text['text'].strip()
                        elif 'sold' in text['text']:
                            sold = text['text'].strip()
                else:
                    available = ''
                    sold = ''
                    
                # = = = = = = = = = = = = = = =
                
                if 'largeImg' in js['o']['w'][0][2]['model']['modules']['PICTURE']['mediaList'][0]['image']:
                    src = js['o']['w'][0][2]['model']['modules']['PICTURE']['mediaList'][0]['image']['largeImg']['URL'].strip()
                else:
                    src = js['o']['w'][0][2]['model']['modules']['PICTURE']['mediaList'][0]['image']['originalImg']['URL'].strip()

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         '序号': '',
                                         'Item_Number': item_number,
                                         'Title': title,
                                         'Subtitle': subtitle,
                                         'Price': price,
                                         'Available': available,
                                         'Sold': sold,
                                         'Vehicle': '',
                                         'Pic': '',
                                         'Url': url,
                                         'Src': src,
                                         'Vehicle_Page': vehicle_page,
                                         'Category_Id': category_id,
                                         'Market_Id': market_id,
                                         'Country': country,
                                         'Language': language,
                                         'Locale': locale}])

                for i, name_value in zip(range(len(specification)), specification):
                    label = specification[name_value]['labels'][0]['textSpans'][0]['text'].strip()
                    if i == 0 and label == 'Condition':
                        df_temp['Condition'] = specification[name_value]['values'][0]['textualDisplays'][0]['textSpans'][0]['text'].strip()
                    else:
                        df_temp[label.lower()] = specification[name_value]['values'][0]['textSpans'][0]['text'].strip()
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(0.5)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Item_Number': item_number}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(item_number +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(12):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./item_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：356

194505417991  <->  [ok] - 剩余数量：344 - 当前时间：11:18:08
194505417773  <->  [ok] - 剩余数量：343 - 当前时间：11:18:08
194505416936  <->  [ok] - 剩余数量：342 - 当前时间：11:18:09
194505416151  <->  [ok] - 剩余数量：341 - 当前时间：11:18:09
194505418352  <->  [ok] - 剩余数量：340 - 当前时间：11:18:09
194505416611  <->  [ok] - 剩余数量：339 - 当前时间：11:18:09
192540786888  <->  [ok] - 剩余数量：338 - 当前时间：11:18:09
194505417519  <->  [ok] - 剩余数量：337 - 当前时间：11:18:09
194505416045  <->  [ok] - 剩余数量：336 - 当前时间：11:18:10
194505418808  <->  [ok] - 剩余数量：335 - 当前时间：11:18:10
194505418206  <->  [ok] - 剩余数量：334 - 当前时间：11:18:10
194505419690  <->  [ok] - 剩余数量：333 - 当前时间：11:18:10
194505416251  <->  [ok] - 剩余数量：332 - 当前时间：11:18:10
194505418552  <->  [ok] - 剩余数量：331 - 当前时间：11:18:10
194505416394  <->  [ok] - 剩余数量：330 - 当前时间：11:18:10
194505419401  <->  [ok] - 剩余数量：329 - 当前时间：11:18:10
194505419836  <->  [ok] - 剩余数量：328 - 当前时间：11:18:11
194505420068  <->  [ok] - 剩余数量：327 - 当前时间：11:18:11
194505420306  <->  [ok] - 剩余数量：326 - 当前时间：11:18:11
194505420468  <->  [ok

192478427246  <->  [ok] - 剩余数量：183 - 当前时间：11:18:48
192725481008  <->  [ok] - 剩余数量：182 - 当前时间：11:18:48
192725483012  <->  [ok] - 剩余数量：181 - 当前时间：11:18:48
192725482165  <->  [ok] - 剩余数量：180 - 当前时间：11:18:49
192725481766  <->  [ok] - 剩余数量：179 - 当前时间：11:18:49
192725483910  <->  [ok] - 剩余数量：178 - 当前时间：11:18:49
192725483666  <->  [ok] - 剩余数量：177 - 当前时间：11:18:50
192725484422  <->  [ok] - 剩余数量：176 - 当前时间：11:18:50
192725484831  <->  [ok] - 剩余数量：175 - 当前时间：11:18:50
192725485567  <->  [ok] - 剩余数量：174 - 当前时间：11:18:50
192725485257  <->  [ok] - 剩余数量：173 - 当前时间：11:18:51
192492591777  <->  [ok] - 剩余数量：172 - 当前时间：11:18:52
192492593977  <->  [ok] - 剩余数量：171 - 当前时间：11:18:53
192492586976  <->  [ok] - 剩余数量：170 - 当前时间：11:18:53
192492595375  <->  [ok] - 剩余数量：169 - 当前时间：11:18:54
192725481422  <->  [ok] - 剩余数量：168 - 当前时间：11:18:55
192492595962  <->  [ok] - 剩余数量：167 - 当前时间：11:18:55
192492597895  <->  [ok] - 剩余数量：166 - 当前时间：11:18:56
192454736877  <->  [ok] - 剩余数量：165 - 当前时间：11:18:56
192492592408  <->  [ok] - 剩余数量：

192461038502  <->  [ok] - 剩余数量：19 - 当前时间：11:19:44
195669362271  <->  [ok] - 剩余数量：18 - 当前时间：11:19:44
192461039242  <->  [ok] - 剩余数量：17 - 当前时间：11:19:45
192461040906  <->  [ok] - 剩余数量：16 - 当前时间：11:19:45
192461039631  <->  [ok] - 剩余数量：15 - 当前时间：11:19:46
195669367960  <->  [ok] - 剩余数量：14 - 当前时间：11:19:47
195669368019  <->  [ok] - 剩余数量：13 - 当前时间：11:19:47
195669368087  <->  [ok] - 剩余数量：12 - 当前时间：11:19:47
195669368150  <->  [ok] - 剩余数量：11 - 当前时间：11:19:48
195669368319  <->  [ok] - 剩余数量：10 - 当前时间：11:19:48
196084651183  <->  [ok] - 剩余数量：9 - 当前时间：11:19:49
195669368239  <->  [ok] - 剩余数量：8 - 当前时间：11:19:49
196084651217  <->  [ok] - 剩余数量：7 - 当前时间：11:19:49
193432073359  <->  [ok] - 剩余数量：6 - 当前时间：11:19:50
193432073431  <->  [ok] - 剩余数量：5 - 当前时间：11:19:51
193432073468  <->  [ok] - 剩余数量：4 - 当前时间：11:19:51
192399272603  <->  [ok] - 剩余数量：3 - 当前时间：11:19:52
192470815384  <->  [ok] - 剩余数量：2 - 当前时间：11:19:55
192470816410  <->  [ok] - 剩余数量：1 - 当前时间：11:19:56
195656999183  <->  [ok] - 剩余数量：0 - 当前时间：11:19:57
1923358086